<a href="https://colab.research.google.com/github/tinugh/Medicine_Recommendation_System/blob/main/Medicine_reccomendation_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
import pickle
from tensorflow.keras.models import load_model

In [2]:

# Load CSV data into a DataFrame
df = pd.read_csv('/content/medical data.csv')

In [3]:
# Drop rows with missing values in 'Symptoms' and 'Medicine' columns
df = df.dropna(subset=['Symptoms','Disease', 'Medicine'])
df


,Name,DateOfBirth,Gender,Symptoms,Causes,Disease,Medicine
0,John Doe,15-05-1980,Male,"Fever, Cough",Viral Infection,Common Cold,"Ibuprofen, Rest"
1,Jane Smith,10-08-1992,Female,"Headache, Fatigue",Stress,Migraine,Sumatriptan
2,Michael Lee,20-02-1975,Male,Shortness of breath,Pollution,Asthma,Albuterol Inhaler
3,Emily Chen,03-11-1988,Female,"Nausea, Vomiting",Food Poisoning,Gastroenteritis,Oral Rehydration
4,Alex Wong,12-06-2001,Male,Sore Throat,Bacterial Infection,Strep Throat,Penicillin
...,...,...,...,...,...,...,...
282,Ava Lim,11-07-1993,Female,"Cough, Shortness of breath",COVID-19 Exposure,COVID-19,"Isolation, Fluids"
283,Liam Goh,16-09-1987,Male,"Itching, Redness",Allergies,Allergic Reaction An,tihistamine
284,Sophie Tan,18-02-1975,Female,"Muscle Pain, Fatigue",Overexertion,Muscle Strain,"Rest, Pain Relievers"
285,Noah Lim,21-05-2005,Male,"Nausea, Vomiting",Food Poisoning,Muscle Strain,Rehydration


In [4]:

# Preprocess text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Disease'].values)
input_sequences = tokenizer.texts_to_sequences(df['Disease'].values)


In [5]:
with open('tokenizer.pickle','wb') as handle:
 pickle.dump(tokenizer,handle,protocol = pickle.HIGHEST_PROTOCOL)

In [6]:
max_sequence_len = max([len(seq) for seq in input_sequences])
vocab_size = len(tokenizer.word_index) + 1

In [7]:
# Pad sequences to a uniform length
input_sequences_padded = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='post')


In [8]:
# Encode target labels (Medicine)
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(df['Medicine'].values)


In [9]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(input_sequences_padded, encoded_labels, test_size=0.2, random_state=42)


In [10]:
#Build an LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_sequence_len))
model.add(LSTM(150))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [11]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

TRAINING THE DATA

In [12]:
# Train the model
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
7/7 [==============================] - 9s 214ms/step - loss: 4.1667 - accuracy: 0.1140
Epoch 2/100
7/7 [==============================] - 1s 106ms/step - loss: 4.1363 - accuracy: 0.3212
Epoch 3/100
7/7 [==============================] - 1s 154ms/step - loss: 4.1018 - accuracy: 0.4404
Epoch 4/100
7/7 [==============================] - 0s 11ms/step - loss: 4.0486 - accuracy: 0.3420
Epoch 5/100
7/7 [==============================] - 1s 123ms/step - loss: 3.9634 - accuracy: 0.2332
Epoch 6/100
7/7 [==============================] - 0s 67ms/step - loss: 3.8232 - accuracy: 0.1865
Epoch 7/100
7/7 [==============================] - 0s 9ms/step - loss: 3.5892 - accuracy: 0.1244
Epoch 8/100
7/7 [==============================] - 0s 8ms/step - loss: 3.3530 - accuracy: 0.1347
Epoch 9/100
7/7 [==============================] - 0s 8ms/step - loss: 3.1694 - accuracy: 0.2332
Epoch 10/100
7/7 [==============================] - 0s 9ms/step - loss: 2.9657 - accuracy: 0.2902
Epoch 11/100
7/7 [=

In [13]:
model.save('medication.h5')

TESTING THE DATA


In [ ]:
# Load the trained model
model = load_model('medication.h5')

# Load the tokenizer
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Define input data
symptoms = input("Enter the symptoms: ")
cause = input("Enter the cause: ")
disease_name = input("Enter the disease name: ")

# Preprocess input data
input_text = f"{symptoms} {disease_name} {cause}"
input_sequence = tokenizer.texts_to_sequences([input_text])
input_padded = pad_sequences(input_sequence, maxlen=max_sequence_len, padding='post')

# Make a prediction
predicted_label = model.predict(input_padded)
predicted_medicine_index = np.argmax(predicted_label)
predicted_medicine = label_encoder.inverse_transform([predicted_medicine_index])[0]

print(f"Predicted Medicine: {predicted_medicine}")

Enter the symptoms: cough.cold
Enter the cause: viral infection
Enter the disease name: common cold
1/1 [==============================] - 0s 362ms/step
Predicted Medicine: Ibuprofen, Rest
